In [29]:
import pandas as pd
from datasets import Dataset
import pickle
from utils import id2label, label2id, classes

In [30]:
id2label

{0: 'dialogues',
 1: 'dissertation',
 2: 'enquiry',
 3: 'essay',
 4: 'history',
 5: 'political discourses',
 6: 'treatise'}

In [31]:
df = pd.read_json('all.json')
history_dataset = df[df.genre=='history']
titles = history_dataset['title'].to_list()
DATA = Dataset.from_pandas(history_dataset)

In [32]:
def predict_logreg(model_path='models/logreg.pkl', vect_path='models/tfidf_vectorizer.pkl'):

    # load
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    with open(vect_path, 'rb') as f:
        vect = pickle.load(f)

    x_test = vect.transform(DATA['text'])
    pred = model.predict_proba(x_test)
    
    return pred

In [33]:
pred_logreg = predict_logreg('models/logreg_stopwords.pkl', 'models/tfidf_vectorizer_stopwords.pkl')

In [38]:
total = 0
new_history = []
i = 0
for sample in pred_logreg:
    if sample[4] > 0.9:
        total += 1
        new_history.append(DATA[i])
    i += 1
        
total

348

In [35]:
DATA[52]

{'title': 'I',
 'genre': 'history',
 'text': "This able prince perished with his son, Osfrid, in a great battle which he fought against Penda, king of Mercia, and Caedwalla, king of the Britons. That event, which happened in the forty-eighth year of Edwin's age and seventeenth of his reign, divided the monarchy of Northumberland, which that prince had united in his person. Eanfrid, the son of Adelfrid, returned with his brothers, Oswald and Oswy, from Scotland, and took possession of Bernicia, his paternal kingdom: Osric, Edwin's cousin-german, established himself in Deïri, the inheritance of his family; but to which the sons of Edwin had a preferable title. Eanfrid, the elder surviving son, fled to Penda, by whom he was treacherously slain. The younger son, Vuscfraea, with Yffi, the grandson of Edwin, by Osfrid, sought protection in Kent, and not finding themselves in safety there, retired into France to King Dagobert, where they died.",
 'labels': 4,
 '__index_level_0__': 2186}

In [45]:
new_history_df = df.from_dict(new_history).rename(columns={'__index_level_0__':'index'}).set_index('index')

In [48]:
new_history_df.to_json('hume_json/history_automatic.json')

In [40]:
history_dataset

,title,genre,text,labels
2134,I,history,"THE CURIOSITY, entertained by all civilized na...",4
2135,I,history,All ancient writers agree in representing the ...,4
2136,I,history,The Britons were divided into many small natio...,4
2137,I,history,The religion of the Britons was one of the mos...,4
2138,I,history,No species of superstition was ever more terri...,4
...,...,...,...,...
7174,LXXI,history,Most of the celebrated writers of this age rem...,4
7175,LXXI,history,The very name of Rochester is offensive to mod...,4
7176,LXXI,history,Wycherley was ambitious of the reputation of w...,4
7177,LXXI,history,"Of all the considerable writers of this age, S...",4
